In [3]:
import numpy as np
import torch
import torch.nn as nn

In [7]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73,200,11],[32,11,43],[99,67,46],[76,56,96],
                  [73,200,11],[32,11,43],[99,67,46],[76,56,96],
                  [73,200,11],[32,11,43],[99,67,46],[76,56,96],
                  [73,200,11],[32,11,43],[99,67,46],[76,56,96]], dtype = 'float32')
# targets (apples, bananas)
targets = np.array([[32,45],[42,55],[86,23],[32,78],
                    [32,45],[42,55],[86,23],[32,78],
                    [32,45],[42,55],[86,23],[32,78],
                    [32,45],[42,55],[86,23],[32,78]], dtype='float32')

In [9]:
inputs =  torch.from_numpy(np.asarray(inputs))
targets= torch.from_numpy(np.asarray(targets))

## Dataset and DataLoader

In [14]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [12]:
# define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:5]

(tensor([[ 73., 200.,  11.],
         [ 32.,  11.,  43.],
         [ 99.,  67.,  46.],
         [ 76.,  56.,  96.],
         [ 73., 200.,  11.]]),
 tensor([[32., 45.],
         [42., 55.],
         [86., 23.],
         [32., 78.],
         [32., 45.]]))

In [20]:
# define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [23]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 76.,  56.,  96.],
        [ 73., 200.,  11.],
        [ 76.,  56.,  96.],
        [ 99.,  67.,  46.],
        [ 99.,  67.,  46.]])
tensor([[32., 78.],
        [32., 45.],
        [32., 78.],
        [86., 23.],
        [86., 23.]])


## nn.Linear 
Instead of initializing the weights and biases manually, we can define using the model nnn.Linear

In [24]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1014,  0.4102, -0.1277],
        [-0.2499, -0.0737,  0.2732]], requires_grad=True)
Parameter containing:
tensor([-0.3415,  0.0269], requires_grad=True)


In [26]:
# parameters 
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1014,  0.4102, -0.1277],
         [-0.2499, -0.0737,  0.2732]], requires_grad=True),
 Parameter containing:
 tensor([-0.3415,  0.0269], requires_grad=True)]

In [28]:
# Generate predictions
preds =  model(inputs)
preds

tensor([[ 87.7020, -29.9523],
        [  1.9241,   2.9665],
        [ 31.3075, -17.0866],
        [ 18.0772,   3.1335],
        [ 87.7020, -29.9523],
        [  1.9241,   2.9665],
        [ 31.3075, -17.0866],
        [ 18.0772,   3.1335],
        [ 87.7020, -29.9523],
        [  1.9241,   2.9665],
        [ 31.3075, -17.0866],
        [ 18.0772,   3.1335],
        [ 87.7020, -29.9523],
        [  1.9241,   2.9665],
        [ 31.3075, -17.0866],
        [ 18.0772,   3.1335]], grad_fn=<AddmmBackward0>)

## Loss Function
Instead of defining a loss function manually, we can use the built in function

In [29]:
import torch.nn.functional as F

In [30]:
loss_fn = F.mse_loss

In [31]:
loss = loss_fn(model(inputs), targets)
loss

tensor(2928.8965, grad_fn=<MseLossBackward0>)

## Optimizer

In [32]:
opt = torch.optim.SGD(model.parameters(),lr = 1e-5)

## Train the model

In [44]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    # Repeat
    for epoch in range(num_epochs):
        # train with batches of data
        for xb, yb in train_dl:
            # pred
            pred = model(xb)
            # loss
            loss = loss_fn(pred, yb)
            # compute gradients
            loss.backward()
            # update parameters
            opt.step()
            # reset gradients to zero
            opt.zero_grad()
            # progress
            if( epoch+1) % 10 == 0:
                print('Epoch [{}/{}, loss: {:.4f}'. format(epoch +1, num_epochs,loss.item()))

In [45]:
fit(100,model, loss_fn, opt)

Epoch [10/100, loss: 353.7819
Epoch [10/100, loss: 344.4095
Epoch [10/100, loss: 251.6725
Epoch [10/100, loss: 468.6475
Epoch [20/100, loss: 232.8800
Epoch [20/100, loss: 390.8830
Epoch [20/100, loss: 265.3271
Epoch [20/100, loss: 200.1847
Epoch [30/100, loss: 277.5852
Epoch [30/100, loss: 226.1246
Epoch [30/100, loss: 319.0843
Epoch [30/100, loss: 25.7434
Epoch [40/100, loss: 227.5175
Epoch [40/100, loss: 223.8382
Epoch [40/100, loss: 275.9221
Epoch [40/100, loss: 8.4530
Epoch [50/100, loss: 198.0411
Epoch [50/100, loss: 114.4330
Epoch [50/100, loss: 262.6055
Epoch [50/100, loss: 442.1393
Epoch [60/100, loss: 188.4854
Epoch [60/100, loss: 128.0979
Epoch [60/100, loss: 263.9757
Epoch [60/100, loss: 223.7314
Epoch [70/100, loss: 147.8586
Epoch [70/100, loss: 88.9623
Epoch [70/100, loss: 258.3140
Epoch [70/100, loss: 439.8669
Epoch [80/100, loss: 226.8313
Epoch [80/100, loss: 223.7977
Epoch [80/100, loss: 102.5539
Epoch [80/100, loss: 4.2813
Epoch [90/100, loss: 162.0154
Epoch [90/100, l

In [48]:
preds = model(inputs)
preds

tensor([[31.2836, 42.8012],
        [21.3139, 34.0864],
        [76.6841, 31.1863],
        [46.3708, 83.3085],
        [31.2836, 42.8012],
        [21.3139, 34.0864],
        [76.6841, 31.1863],
        [46.3708, 83.3085],
        [31.2836, 42.8012],
        [21.3139, 34.0864],
        [76.6841, 31.1863],
        [46.3708, 83.3085],
        [31.2836, 42.8012],
        [21.3139, 34.0864],
        [76.6841, 31.1863],
        [46.3708, 83.3085]], grad_fn=<AddmmBackward0>)

In [49]:
print(targets)

tensor([[32., 45.],
        [42., 55.],
        [86., 23.],
        [32., 78.],
        [32., 45.],
        [42., 55.],
        [86., 23.],
        [32., 78.],
        [32., 45.],
        [42., 55.],
        [86., 23.],
        [32., 78.],
        [32., 45.],
        [42., 55.],
        [86., 23.],
        [32., 78.]])
